In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

In [3]:
import json

In [4]:
df_viajes=pd.read_csv("viajes.csv")

In [5]:
df_viajes.head()

,id,city,country,region,short_description,latitude,longitude,avg_temp_monthly,ideal_durations,budget_level,culture,adventure,nature,beaches,nightlife,cuisine,wellness,urban,seclusion
0,c54acf38-3029-496b-8c7a-8343ad82785c,Milan,Italy,europe,"Chic streets lined with fashion boutiques, his...",45.464194,9.189635,"{""1"":{""avg"":3.7,""max"":7.8,""min"":0.4},""2"":{""avg...","[""Short trip"",""One week""]",Luxury,5,2,2,1,4,5,3,5,2
1,0bd12654-ed64-424e-a044-7bc574bcf078,Yasawa Islands,Fiji,oceania,"Crystal-clear waters, secluded beaches, and vi...",-17.290947,177.125786,"{""1"":{""avg"":28,""max"":30.8,""min"":25.8},""2"":{""av...","[""Long trip"",""One week""]",Luxury,2,4,5,5,2,3,4,1,5
2,73036cda-9134-46fc-a2c6-807782d59dfb,Whistler,Canada,north_america,Snow-capped peaks and lush forests create a se...,50.117190,-122.954302,"{""1"":{""avg"":-2.5,""max"":0.4,""min"":-5.5},""2"":{""a...","[""Short trip"",""Weekend"",""One week""]",Luxury,3,5,5,2,3,3,4,2,4
3,3872c9c0-6b6e-49e1-9743-f46bfe591b86,Guanajuato,Mexico,north_america,Winding cobblestone streets and colorful facad...,20.987700,-101.000000,"{""1"":{""avg"":15.5,""max"":22.8,""min"":8.7},""2"":{""a...","[""Weekend"",""One week"",""Short trip""]",Mid-range,5,3,3,1,3,4,3,4,2
4,e1ebc1b6-8798-422d-847a-22016faff3fd,Surabaya,Indonesia,asia,Bustling streets filled with the aroma of loca...,-7.245972,112.737827,"{""1"":{""avg"":28.1,""max"":32.5,""min"":25.5},""2"":{""...","[""Short trip"",""Weekend""]",Budget,4,3,3,2,3,4,3,4,2


In [6]:
df_viajes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 560 non-null    object 
 1   city               560 non-null    object 
 2   country            560 non-null    object 
 3   region             560 non-null    object 
 4   short_description  560 non-null    object 
 5   latitude           560 non-null    float64
 6   longitude          560 non-null    float64
 7   avg_temp_monthly   560 non-null    object 
 8   ideal_durations    560 non-null    object 
 9   budget_level       560 non-null    object 
 10  culture            560 non-null    int64  
 11  adventure          560 non-null    int64  
 12  nature             560 non-null    int64  
 13  beaches            560 non-null    int64  
 14  nightlife          560 non-null    int64  
 15  cuisine            560 non-null    int64  
 16  wellness           560 non

In [7]:
df_viajes.shape

(560, 19)

In [37]:
for col in df_viajes.select_dtypes(include='number'):
    print('-----------------------------')
    print(f"La distribución de las categorías para la columna {col.upper()}")
    print(df_viajes[col].nunique())
    print(df_viajes[col].value_counts(normalize=True)) 

-----------------------------
La distribución de las categorías para la columna LATITUDE
552
latitude
 50.061947    0.003571
 38.895037    0.003571
 40.712728    0.003571
 64.145981    0.003571
 46.813743    0.003571
                ...   
-19.986095    0.001786
 57.707233    0.001786
 53.479489    0.001786
 55.686724    0.001786
-20.354626    0.001786
Name: proportion, Length: 552, dtype: float64
-----------------------------
La distribución de las categorías para la columna LONGITUDE
552
longitude
 19.936856     0.003571
-77.036543     0.003571
-74.006015     0.003571
-21.942237     0.003571
-71.208406     0.003571
                 ...   
 23.422435     0.001786
 11.967017     0.001786
-2.245115      0.001786
 12.570072     0.001786
 148.956052    0.001786
Name: proportion, Length: 552, dtype: float64
-----------------------------
La distribución de las categorías para la columna CULTURE
4
culture
4    0.425000
3    0.308929
5    0.230357
2    0.035714
Name: proportion, dtype: float6

In [8]:
for col in df_viajes.select_dtypes(include='object'):
    print('-----------------------------')
    print(f"La distribución de las categorías para la columna {col.upper()}")
    print(df_viajes[col].nunique())
    print(df_viajes[col].value_counts(normalize=True)) 

-----------------------------
La distribución de las categorías para la columna ID
560
id
62caf451-c399-4d64-a770-1efd93a603ef    0.001786
c54acf38-3029-496b-8c7a-8343ad82785c    0.001786
0bd12654-ed64-424e-a044-7bc574bcf078    0.001786
73036cda-9134-46fc-a2c6-807782d59dfb    0.001786
3872c9c0-6b6e-49e1-9743-f46bfe591b86    0.001786
                                          ...   
630b0ecb-8651-476e-803d-b65f99f342a3    0.001786
0d1ac2c5-e3d5-40b5-ad56-01f3d67710ab    0.001786
955899e4-7ccb-4b6a-a85c-a9b73efe6876    0.001786
57239ab4-5376-4b81-a35e-fc8a27fb5812    0.001786
6d8f232e-d34d-4dac-95c4-e01d7983124c    0.001786
Name: proportion, Length: 560, dtype: float64
-----------------------------
La distribución de las categorías para la columna CITY
559
city
Granada       0.003571
Guanajuato    0.001786
Surabaya      0.001786
Nuuk          0.001786
Windhoek      0.001786
                ...   
Maun          0.001786
Gothenburg    0.001786
Manchester    0.001786
Copenhagen    0.001786
W

In [9]:
df_viajes['avg_temp_monthly'] = df_viajes['avg_temp_monthly'].apply(json.loads)

In [10]:
rows = []
for i, row in df_viajes.iterrows():
    id_val = row['id']
    monthly_dict = row['avg_temp_monthly']
    for month, temps in monthly_dict.items():
        rows.append({
            'id': id_val,
            'Month': int(month),
            'Avg': temps['avg'],
            'Max': temps['max'],
            'Min': temps['min']
        })

df_monthly = pd.DataFrame(rows)

In [11]:
df_monthly.head()

,id,Month,Avg,Max,Min
0,c54acf38-3029-496b-8c7a-8343ad82785c,1,3.7,7.8,0.4
1,c54acf38-3029-496b-8c7a-8343ad82785c,2,7.1,12.0,2.8
2,c54acf38-3029-496b-8c7a-8343ad82785c,3,10.5,15.5,5.5
3,c54acf38-3029-496b-8c7a-8343ad82785c,4,13.8,18.9,8.7
4,c54acf38-3029-496b-8c7a-8343ad82785c,5,17.9,22.5,13.4


In [28]:
df_monthly.describe().T

,count,mean,std,min,25%,50%,75%,max
Month,6720.0,6.500000,3.452309,1.0,3.75,6.50,9.25,12.0
Avg,6720.0,17.966071,8.934916,-22.1,11.80,19.30,25.80,40.2
Max,6720.0,22.688958,9.257218,-14.8,16.70,24.55,30.20,47.0
Min,6720.0,13.747426,9.003137,-23.7,7.20,14.50,21.70,33.4


In [12]:
df_monthly.shape

(6720, 5)

In [31]:
df_monthly['Min'] = df_monthly['Min'].astype(str).str.replace('.', ',', regex=False)

In [34]:
df_monthly['Max'] = df_monthly['Max'].astype(str).str.replace('.', ',', regex=False)

In [35]:
df_monthly['Avg'] = df_monthly['Avg'].astype(str).str.replace('.', ',', regex=False)

In [32]:
df_monthly.head()

,id,Month,Avg,Max,Min
0,c54acf38-3029-496b-8c7a-8343ad82785c,1,3.7,7.8,"0,4"
1,c54acf38-3029-496b-8c7a-8343ad82785c,2,7.1,12.0,"2,8"
2,c54acf38-3029-496b-8c7a-8343ad82785c,3,10.5,15.5,"5,5"
3,c54acf38-3029-496b-8c7a-8343ad82785c,4,13.8,18.9,"8,7"
4,c54acf38-3029-496b-8c7a-8343ad82785c,5,17.9,22.5,"13,4"


In [36]:
df_monthly.to_csv("monthly_temp.csv",index=False)

In [17]:
# Convertir la columna de listas como texto a listas reales
df_viajes['ideal_durations_list'] = df_viajes['ideal_durations'].apply(lambda x: json.loads(x))
# Definir las duraciones a evaluar
durations = ["Weekend", "Short trip", "One week", "Long trip", "Day trip"]
# Crear columnas con "Sí" o "No" por cada duración
for duration in durations:
    col_name = f'{duration.lower().replace(" ", "_")}'
    df_viajes[col_name] = df_viajes['ideal_durations_list'].apply(
        lambda x: "Sí" if duration in x else "No"
    )
# Crear una tabla con solo la ciudad y las nuevas columnas de duración
duration_flags = df_viajes[['id'] + [f'{d.lower().replace(" ", "_")}' for d in durations]]

In [18]:
df_viajes.head()

,id,city,country,region,short_description,latitude,longitude,avg_temp_monthly,ideal_durations,budget_level,culture,adventure,nature,beaches,nightlife,cuisine,wellness,urban,seclusion,ideal_durations_list,duration_weekend,duration_short_trip,duration_one_week,duration_long_trip,duration_day_trip,weekend,short_trip,one_week,long_trip,day_trip
0,c54acf38-3029-496b-8c7a-8343ad82785c,Milan,Italy,europe,"Chic streets lined with fashion boutiques, his...",45.464194,9.189635,"{'1': {'avg': 3.7, 'max': 7.8, 'min': 0.4}, '2...","[""Short trip"",""One week""]",Luxury,5,2,2,1,4,5,3,5,2,"[Short trip, One week]",No,Sí,Sí,No,No,No,Sí,Sí,No,No
1,0bd12654-ed64-424e-a044-7bc574bcf078,Yasawa Islands,Fiji,oceania,"Crystal-clear waters, secluded beaches, and vi...",-17.290947,177.125786,"{'1': {'avg': 28, 'max': 30.8, 'min': 25.8}, '...","[""Long trip"",""One week""]",Luxury,2,4,5,5,2,3,4,1,5,"[Long trip, One week]",No,No,Sí,Sí,No,No,No,Sí,Sí,No
2,73036cda-9134-46fc-a2c6-807782d59dfb,Whistler,Canada,north_america,Snow-capped peaks and lush forests create a se...,50.117190,-122.954302,"{'1': {'avg': -2.5, 'max': 0.4, 'min': -5.5}, ...","[""Short trip"",""Weekend"",""One week""]",Luxury,3,5,5,2,3,3,4,2,4,"[Short trip, Weekend, One week]",Sí,Sí,Sí,No,No,Sí,Sí,Sí,No,No
3,3872c9c0-6b6e-49e1-9743-f46bfe591b86,Guanajuato,Mexico,north_america,Winding cobblestone streets and colorful facad...,20.987700,-101.000000,"{'1': {'avg': 15.5, 'max': 22.8, 'min': 8.7}, ...","[""Weekend"",""One week"",""Short trip""]",Mid-range,5,3,3,1,3,4,3,4,2,"[Weekend, One week, Short trip]",Sí,Sí,Sí,No,No,Sí,Sí,Sí,No,No
4,e1ebc1b6-8798-422d-847a-22016faff3fd,Surabaya,Indonesia,asia,Bustling streets filled with the aroma of loca...,-7.245972,112.737827,"{'1': {'avg': 28.1, 'max': 32.5, 'min': 25.5},...","[""Short trip"",""Weekend""]",Budget,4,3,3,2,3,4,3,4,2,"[Short trip, Weekend]",Sí,Sí,No,No,No,Sí,Sí,No,No,No


In [20]:
df_viajes.drop(columns=["avg_temp_monthly", "ideal_durations", "ideal_durations_list", "duration_weekend", "duration_short_trip", "duration_one_week", "duration_long_trip","duration_day_trip"],inplace=True)

In [21]:
df_viajes.head()

,id,city,country,region,short_description,latitude,longitude,budget_level,culture,adventure,nature,beaches,nightlife,cuisine,wellness,urban,seclusion,weekend,short_trip,one_week,long_trip,day_trip
0,c54acf38-3029-496b-8c7a-8343ad82785c,Milan,Italy,europe,"Chic streets lined with fashion boutiques, his...",45.464194,9.189635,Luxury,5,2,2,1,4,5,3,5,2,No,Sí,Sí,No,No
1,0bd12654-ed64-424e-a044-7bc574bcf078,Yasawa Islands,Fiji,oceania,"Crystal-clear waters, secluded beaches, and vi...",-17.290947,177.125786,Luxury,2,4,5,5,2,3,4,1,5,No,No,Sí,Sí,No
2,73036cda-9134-46fc-a2c6-807782d59dfb,Whistler,Canada,north_america,Snow-capped peaks and lush forests create a se...,50.117190,-122.954302,Luxury,3,5,5,2,3,3,4,2,4,Sí,Sí,Sí,No,No
3,3872c9c0-6b6e-49e1-9743-f46bfe591b86,Guanajuato,Mexico,north_america,Winding cobblestone streets and colorful facad...,20.987700,-101.000000,Mid-range,5,3,3,1,3,4,3,4,2,Sí,Sí,Sí,No,No
4,e1ebc1b6-8798-422d-847a-22016faff3fd,Surabaya,Indonesia,asia,Bustling streets filled with the aroma of loca...,-7.245972,112.737827,Budget,4,3,3,2,3,4,3,4,2,Sí,Sí,No,No,No


In [22]:
df_viajes['region'] = df_viajes['region'].str.replace('_', ' ').str.title()

In [23]:
df_viajes.head()

,id,city,country,region,short_description,latitude,longitude,budget_level,culture,adventure,nature,beaches,nightlife,cuisine,wellness,urban,seclusion,weekend,short_trip,one_week,long_trip,day_trip
0,c54acf38-3029-496b-8c7a-8343ad82785c,Milan,Italy,Europe,"Chic streets lined with fashion boutiques, his...",45.464194,9.189635,Luxury,5,2,2,1,4,5,3,5,2,No,Sí,Sí,No,No
1,0bd12654-ed64-424e-a044-7bc574bcf078,Yasawa Islands,Fiji,Oceania,"Crystal-clear waters, secluded beaches, and vi...",-17.290947,177.125786,Luxury,2,4,5,5,2,3,4,1,5,No,No,Sí,Sí,No
2,73036cda-9134-46fc-a2c6-807782d59dfb,Whistler,Canada,North America,Snow-capped peaks and lush forests create a se...,50.117190,-122.954302,Luxury,3,5,5,2,3,3,4,2,4,Sí,Sí,Sí,No,No
3,3872c9c0-6b6e-49e1-9743-f46bfe591b86,Guanajuato,Mexico,North America,Winding cobblestone streets and colorful facad...,20.987700,-101.000000,Mid-range,5,3,3,1,3,4,3,4,2,Sí,Sí,Sí,No,No
4,e1ebc1b6-8798-422d-847a-22016faff3fd,Surabaya,Indonesia,Asia,Bustling streets filled with the aroma of loca...,-7.245972,112.737827,Budget,4,3,3,2,3,4,3,4,2,Sí,Sí,No,No,No


In [26]:
df_viajes.to_csv("travel_info.csv",index=False)